importing all relevant libraries

In [20]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

reading in csv file containing all the relevent features we are interested in to explore and find a model

In [9]:
df = pd.read_csv('our_dataframe.csv', index_col=0)

In [10]:
df.columns

Index(['saleprice', 'sqfttotliving', 'yrbuilt', 'yrrenovated', 'bedrooms',
       'zipcode', 'sqfttotbasement', 'sqftfinbasement', 'sqftopenporch',
       'sqftenclosedporch', 'sqftdeck', 'heatsystem', 'heatsource',
       'bathhalfcount', 'bath3qtrcount', 'bathfullcount', 'condition',
       'viewutilization', 'sqftgarageattached', 'daylightbasement',
       'finbasementgrade', 'hbuasifvacant', 'inadequateparking', 'mtrainier',
       'olympics', 'cascades', 'territorial', 'seattleskyline', 'pugetsound',
       'lakewashington', 'lakesammamish', 'smalllakerivercreek', 'otherview',
       'wfntlocation', 'trafficnoise', 'airportnoise', 'powerlines',
       'othernuisances', 'adjacentgreenbelt'],
      dtype='object')

Adding in bath, porch, price per square foot in data frame and dropping individual columns containing parts of these newly added columns

In [11]:
df['porch'] = df['sqftopenporch'] + df['sqftenclosedporch']
df['baths'] = df['bathfullcount'] + (0.75*df['bath3qtrcount']) + (0.5*df['bathhalfcount'])
df.drop(columns=['sqftopenporch', 'sqftenclosedporch', 'bathfullcount', 'bath3qtrcount',
                 'bathhalfcount'], inplace=True)

In [12]:
df['pricepersqft'] = df['saleprice']/df['sqfttotliving']
df['pricepersqft_finbasement'] = df['saleprice']/ (df['sqfttotliving'] + df['sqftfinbasement'])

In [32]:
df.shape

(18771, 38)

In [50]:
df['saleprice']

0         560000
1         435000
2         409950
3         380000
4         775000
          ...   
18766     930000
18767     850000
18768     686000
18769     695000
18770    1550000
Name: saleprice, Length: 18771, dtype: int64

In [33]:
df.columns

Index(['saleprice', 'sqfttotliving', 'yrbuilt', 'yrrenovated', 'bedrooms',
       'zipcode', 'sqfttotbasement', 'sqftfinbasement', 'sqftdeck',
       'heatsystem', 'heatsource', 'condition', 'viewutilization',
       'sqftgarageattached', 'daylightbasement', 'finbasementgrade',
       'hbuasifvacant', 'inadequateparking', 'mtrainier', 'olympics',
       'cascades', 'territorial', 'seattleskyline', 'pugetsound',
       'lakewashington', 'lakesammamish', 'smalllakerivercreek', 'otherview',
       'wfntlocation', 'trafficnoise', 'airportnoise', 'powerlines',
       'othernuisances', 'adjacentgreenbelt', 'porch', 'baths', 'pricepersqft',
       'pricepersqft_finbasement'],
      dtype='object')

In [40]:
df_corr = df.corr()['saleprice'].sort_values(ascending=False)

In [42]:
len(df_corr)

32

In [49]:
df_corr[abs(df_corr) > 0.2]

saleprice                   1.000000
sqfttotliving               0.621515
baths                       0.495995
pricepersqft                0.480202
pricepersqft_finbasement    0.415050
lakewashington              0.378633
territorial                 0.312686
sqftfinbasement             0.291167
sqfttotbasement             0.272623
bedrooms                    0.271100
porch                       0.270791
wfntlocation                0.261991
finbasementgrade            0.234935
sqftdeck                    0.208910
Name: saleprice, dtype: float64

choosing the highest correlation with saleprice and also choosing only the features of the property for now (will look into the location aspects which is the key in real estate as we progress)

sqfttotliving 0.621515 vs pricepersqft 0.480202 vs pricepersqft_finbasement 0.415050 (essentially the same thing in different forms, can include the one that best suits the R-squareed and works with other predictors)

baths 0.495995

sqftfinbasement 0.291167, sqfttotbasement 0.272623

bedrooms  0.271100

porch 0.270791

lakewashington view 0.378633, territorial view 0.312686 (the codes used in these two range from 1-4 relating to fair - excellent view)

wfntlocation 0.261991 (this should be more of a yes and no type field since the codes used for this variable only categorizes a unit having 1 of 9 different waterfront areas in king county, need to look into this more carefully)

finished basement grade (1-20 relating to )

checking for linearity between the chosen predictors

In [48]:
df['viewutilization'].value_counts()

     9625
N    8721
Y     425
Name: viewutilization, dtype: int64